In [1]:
#Chatbot Training Model
import os
import pyttsx3
import time
import playsound
import speech_recognition as sr
from gtts import gTTS
from spellchecker import SpellChecker
from textblob import TextBlob
import emoji
import time
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('cap_intents.json',encoding="utf8").read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


152 documents
39 classes ['Age_group', 'Cloud_features', 'Experiences', 'Impact', 'Inner_explorer', 'Language', 'Mindfulness', 'Mindfulness_Programs', 'Mobile_app', 'Name', 'Need_mindfulness', 'Trial_audio', 'Well_being', 'advicefor_child', 'all_cost', 'app_availability', 'appreciation', 'audio_language', 'bad', 'chatbot', 'cloud_cost', 'community_cost', 'district_cost', 'download_app', 'goodbye', 'greeting', 'no_access_student', 'noanswer', 'parent', 'practice-detail', 'program_advice', 'school district', 'schools', 'single_school', 'smart', 'steps', 'support', 'thanks', 'work']
199 unique lemmatized words ["'m", "'s", "'ve", ',', '.', '1', '13', '2', '3', 'a', 'achievement', 'actually', 'additional', 'advantage', 'age-group', 'agenda', 'aggressive', 'all', 'am', 'amazing', 'an', 'and', 'answer', 'any', 'anyone', 'app', 'are', 'at', 'audio', 'availability', 'available', 'average', 'awesome', 'be', 'best', 'brief', 'but', 'bye', 'can', 'chatting', 'child', 'choose', 'clear', 'cloud', '

C:\Users\DELL\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Epoch 1/200
31/31 [==============================] - 0s 1ms/step - loss: 3.6249 - accuracy: 0.0243 
Epoch 2/200
31/31 [==============================] - 0s 1ms/step - loss: 3.5872 - accuracy: 0.0769
Epoch 3/200
31/31 [==============================] - 0s 1ms/step - loss: 3.5570 - accuracy: 0.0771
Epoch 4/200
31/31 [==============================] - 0s 1ms/step - loss: 3.3250 - accuracy: 0.1453
Epoch 5/200
31/31 [==============================] - 0s 1ms/step - loss: 3.4807 - accuracy: 0.0638
Epoch 6/200
31/31 [==============================] - 0s 1ms/step - loss: 3.1548 - accuracy: 0.1603
Epoch 7/200
31/31 [==============================] - 0s 1ms/step - loss: 3.0036 - accuracy: 0.2262
Epoch 8/200
31/31 [==============================] - 0s 1ms/step - loss: 2.8551 - accuracy: 0.2711
Epoch 9/200
31/31 [==============================] - 0s 1ms/step - loss: 2.7684 - accuracy: 0.3021
Epoch 10/200
31/31 [==============================] - 0s 1ms/step - loss: 2.4941 - accuracy: 0.2965
Epoch 11

31/31 [==============================] - 0s 1ms/step - loss: 0.2987 - accuracy: 0.9242
Epoch 84/200
31/31 [==============================] - 0s 1ms/step - loss: 0.2257 - accuracy: 0.9317
Epoch 85/200
31/31 [==============================] - 0s 1ms/step - loss: 0.1960 - accuracy: 0.9167
Epoch 86/200
31/31 [==============================] - 0s 1ms/step - loss: 0.2880 - accuracy: 0.9085
Epoch 87/200
31/31 [==============================] - 0s 1ms/step - loss: 0.2665 - accuracy: 0.9395
Epoch 88/200
31/31 [==============================] - 0s 1ms/step - loss: 0.4204 - accuracy: 0.9066
Epoch 89/200
31/31 [==============================] - 0s 1ms/step - loss: 0.3230 - accuracy: 0.8833
Epoch 90/200
31/31 [==============================] - 0s 1ms/step - loss: 0.2963 - accuracy: 0.8651
Epoch 91/200
31/31 [==============================] - 0s 1ms/step - loss: 0.2893 - accuracy: 0.8863
Epoch 92/200
31/31 [==============================] - 0s 1ms/step - loss: 0.2234 - accuracy: 0.9558
Epoch 93/200


31/31 [==============================] - 0s 1ms/step - loss: 0.1842 - accuracy: 0.9491
Epoch 165/200
31/31 [==============================] - 0s 1ms/step - loss: 0.1394 - accuracy: 0.9493
Epoch 166/200
31/31 [==============================] - 0s 1ms/step - loss: 0.2571 - accuracy: 0.9219
Epoch 167/200
31/31 [==============================] - 0s 1ms/step - loss: 0.1658 - accuracy: 0.9344
Epoch 168/200
31/31 [==============================] - 0s 1ms/step - loss: 0.1236 - accuracy: 0.9569
Epoch 169/200
31/31 [==============================] - 0s 1ms/step - loss: 0.0978 - accuracy: 0.9751
Epoch 170/200
31/31 [==============================] - 0s 1ms/step - loss: 0.1215 - accuracy: 0.9516
Epoch 171/200
31/31 [==============================] - 0s 1ms/step - loss: 0.2060 - accuracy: 0.9632
Epoch 172/200
31/31 [==============================] - 0s 1ms/step - loss: 0.1128 - accuracy: 0.9788
Epoch 173/200
31/31 [==============================] - 0s 1ms/step - loss: 0.1050 - accuracy: 0.9724
Epoc

In [2]:
#Chatbot API

from spellchecker import SpellChecker
import random
import multiprocessing
from flask import Flask,json, render_template, request
from flask_cors import CORS
import time
import nltk
from multiprocessing import Process
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import json
import numpy as np
from keras.models import load_model
model = load_model('chatbot_model.h5')
intents = json.loads(open('cap_intents.json',encoding="utf8").read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
app= Flask(__name__)
CORS(app)
# @app.route('/run/<msg>') 
def chatbot(msg):
    msg=SpellChecker().correction(msg)
    
    def clean_up_sentence(sentence):
        # token,aize the pattern - split words into array
        sentence_words = nltk.word_tokenize(sentence)
        # stem each word - create short form for word
        sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
        return sentence_words
    # return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
    def bow(sentence, words, show_details=True):
        # tokenize the pattern
        sentence_words = clean_up_sentence(sentence)
        # bag of words - matrix of N words, vocabulary matrix
        bag = [0]*len(words)  
        for s in sentence_words:
            for i,w in enumerate(words):
                if w == s: 
                    # assign 1 if current word is in the vocabulary position
                    bag[i] = 1
                    if show_details:
                        print ("found in bag: %s" % w)
        return(np.array(bag))
    def predict_class(sentence, model):
        # filter out predictions below a threshold
        p = bow(sentence, words,show_details=False)
        
        res = model.predict(np.array([p]))[0]
        ERROR_THRESHOLD = 0.25
        results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
        # sort by strength of probability
        results.sort(key=lambda x: x[1], reverse=True)
        return_list = []
        for r in results:
            return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
        return return_list 
    def getResponse(ints, intents_json):
        tag = ints[0]['intent']
        list_of_intents = intents_json['intents']
        for i in list_of_intents:
            if(i['tag']== tag):
                result = random.choice(i['responses'])
                #result1 = random.choice(i['responses1'])
                break
        return result
    def chatbot_response(text):
        ints = predict_class(text, model)
        res = getResponse(ints, intents)
        return res
    
    def send():
        print(msg,"input")
        while True:#it used to be in continunation and to stop it press alt+cntrl+c
            #msg=input("You:- ")
            
            if msg=="quit":
                break
            else:
                res=chatbot_response(msg)
                
                return {"Bot":[res]}
    return send()
#     if __name__ == '__main__':
    p = Process(target=chatbot)
    p.start()
    p.join()
# if __name__== "__main__":
#     app.run(host='172.31.52.129',port=5000)
    

In [3]:
chatbot("stupid?")

stupid input


{'Bot': ['Sometimes I make mistakes😞, but I can learn from them.\nCan you ask question again?']}

In [4]:
from spellchecker import SpellChecker
import random
import multiprocessing
from flask import Flask,json, render_template, request
from flask_cors import CORS
import time
import nltk
from multiprocessing import Process
from nltk.stem import WordNetLemmatizer
from tensorflow.python.keras.backend import reset_uids
lemmatizer = WordNetLemmatizer()
import pickle
import json
import datetime
import numpy as np
import os
import pyttsx3
import time
import playsound
import speech_recognition as sr
from gtts import gTTS
from keras.models import load_model
model = load_model('chatbot_model.h5')
intents = json.loads(open('cap_intents.json',encoding="utf8").read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
currentTime = datetime.datetime.now()
currentTime.hour
app= Flask(__name__)
CORS(app)
# @app.route('/run') 
def greeting():
    if currentTime.hour < 12:
        return {"Bot": ['Hi,Good Morning! Genie at service.']}
    elif 12 <= currentTime.hour < 18:
        return {"Bot": ['Hi,Good Afternoon! Genie at service.']}
    else:
        return {"Bot": ['Hi,Good evening! Genie at service.']}
@app.route('/run/<msg>') 
def chatbot(msg):
    msg=SpellChecker().correction(msg)
    
    def clean_up_sentence(sentence):
        # token,aize the pattern - split words into array
        sentence_words = nltk.word_tokenize(sentence)
        # stem each word - create short form for word
        sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
        return sentence_words
    # return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
    def bow(sentence, words, show_details=True):
        # tokenize the pattern
        sentence_words = clean_up_sentence(sentence)
        # bag of words - matrix of N words, vocabulary matrix
        bag = [0]*len(words)  
        for s in sentence_words:
            for i,w in enumerate(words):
                if w == s: 
                    # assign 1 if current word is in the vocabulary position
                    bag[i] = 1
                    if show_details:
                        print ("found in bag: %s" % w)
        return(np.array(bag))
    def predict_class(sentence, model):
        # filter out predictions below a threshold
        p = bow(sentence, words,show_details=False)
        
        res = model.predict(np.array([p]))[0]
        ERROR_THRESHOLD = 0.25
        results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
        # sort by strength of probability
        results.sort(key=lambda x: x[1], reverse=True)
        return_list = []
        for r in results:
            return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
        return return_list 
        print(return_list )
    
    def getResponse(ints, intents_json):
        tag = ints[0]['intent']
        list_of_intents = intents_json['intents']
        for i in list_of_intents:
            if(i['tag']== tag):
                result = random.choice(i['responses'])
                #result1 = random.choice(i['responses1'])
                break
        return result
        
    
    def chatbot_response(text):
        ints = predict_class(text, model)
        res = getResponse(ints, intents)
        return res
    
    
#     engine = pyttsx3.init('nsss')
    engine=pyttsx3.init('sapi5')
    voices=engine.getProperty('voices')
    voice_id = "com.apple.speech.synthesis.voice.samantha"
#     engine.setProperty('voice',voice_id)
    engine.setProperty('voice','voices[0].id')
    def speak(text):
        engine.say(text)
        engine.runAndWait()
    def get_audio():
        r = sr.Recognizer()
        with sr.Microphone() as source:
            audio = r.listen(source)
            said = ""
            try:
                said = r.recognize_google(audio)
                print(said)
            except Exception as e:
                print("Exception: " + str(e))
        return said
    def send():
        print(msg,"input")
        while True:#it used to be in continunation and to stop it press alt+cntrl+c
            #msg=input("You:- ")
            
            if msg=="quit":
                break
            else:
                res=chatbot_response(msg)
                prob=predict_class(msg, model)
            
                
                return {"Bot":[res],"probability":[prob]}, speak(res)
                
                
                # return {"Bot":[res]}
    return send()
    
        
    if __name__ == '__main__':
        p = Process(target=chatbot)
        p.start()
        p.join()
# if __name__== "__main__":
#     app.run(threaded=False,port=5000)
    # app.run(host='172.31.52.129',port=5000)

In [5]:
chatbot("hello")

hello input


({'Bot': ["Hi there! I'm GENIE. How Can I help you?😊"],
  'probability': [[{'intent': 'greeting', 'probability': '0.9997259'}]]},
 None)